In [79]:
import sys
import os
import requests
import json
import cnfg
from http import HTTPStatus
from bs4 import BeautifulSoup
import re

In [47]:
config = cnfg.load(os.path.join(os.getenv('HOME'),
                                '.goodreads', 'goodreads_config'))

In [48]:
config

{'app': 'scifiadikt',
 'company': 'home sweet home',
 'key': 'RADX6GCws06x2ZWsZC8wA',
 'secret': 'lnaeoAUDVenbuZlUHTX5gUfCOYLcjKUKzWZOjtrE',
 'user_id': '18694580'}

In [160]:
def make_request(url, timeout=10, retries=3, pause_between_retries=5):
    for n in range(retries):
        if n and pause_between_retries > 0:
            time.sleep(pause_between_retries)
        try:
            response = requests.get(url, timeout=timeout)
            if response.status_code == HTTPStatus.OK:
                return response
        except (requests.exceptions.RequestException) as e:
            print("{}: {}".format(url, e), file=sys.stderr)
            pass

In [124]:
GOOD_READS_ROOT_URL = 'https://www.goodreads.com'
BOOK_BY_ISBN_URL = 'https://www.goodreads.com/book/isbn'

def get_book_reviews(isbn):
    response = make_request('{}/{}?key={}'.format(
        BOOK_BY_ISBN_URL,
        isbn,
        config['key']
    ))
    return response



In [150]:
BLACK_STAR = chr(9733)
WHITE_STAR = chr(9734)

def stars_to_num(rating):
    return sum(map(lambda c: 1 if c == BLACK_STAR else 0, rating.strip()))

def scrape_reviews_page(page):
    soup = BeautifulSoup(page, "lxml")
    
    reviews = []
    for review in soup.find_all('div', class_='gr_review_container'):
        review_rating = review.find('span', class_='gr_rating')
        if review_rating:
            review_rating = stars_to_num(review_rating.text)
        review_date = review.find('span', class_='gr_review_date')
        if review_date:
            review_date = review_date.text.strip()
        review_summary, full_review_url = None, None
        review_text = review.find('div', class_='gr_review_text')
        if review_text:
            review_summary = re.sub(r'\.+\s*\.+more', '', review_text.text.strip())
            review_url = review_text.find('link')
            if review_url:
                full_review_url = review_url.get('href')
        reviews.append([review_rating, review_date, review_summary, full_review_url])
    
    next_page = None
    pagination = soup.find('div', class_='gr_pagination')
    if pagination:
        next_page = pagination.find('a', class_="next_page")
        if next_page:
            next_page = next_page.get('href')
    return reviews, next_page

def scrape_full_review_page(page):
    soup = BeautifulSoup(page, "lxml")
    review = soup.find('div', itemprop='reviewBody')
    if review:
        return review.text

def get_full_review(url):
    response = make_request(url)
    if response:
        return scrape_full_review_page(response.text)
    
def get_reviews(isbn, full_review=False):
    all_reviews = []
    
    next_page = '/api/reviews_widget_iframe?isbn={}'.format(isbn)
    while next_page:
        response = make_request(GOOD_READS_ROOT_URL + next_page)
        if not response:
            break
        reviews, next_page = scrape_reviews_page(response.text)
        if reviews:
            all_reviews.extend(reviews)
    if full_review:
        for review in all_reviews:
            if review[3]:
                review.append(get_full_review(review[3]))
    return all_reviews

In [154]:
%%time
reviews = get_reviews('0441014186', full_review=True)

CPU times: user 4.5 s, sys: 141 ms, total: 4.64 s
Wall time: 1min 34s


In [156]:
len(reviews)

100

In [161]:
response = make_request('https://www.goodreads.com/book/show/112292')

In [162]:
print(response.text)

<!DOCTYPE html>
<html class="desktop pageskin
">


<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">
  <title>
Dauntless (The Lost Fleet, #1) by Jack Campbell
</title>


    <script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();
 </script>
  <script type="text/javascript">
    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(m,l,h,j,i){var k={m:m,f:l,l:h,c:""+j,err:i,fromOnError:1,args:arguments};c.ueLogError(k);return false}b.skipTrace=1;e.onerror=b;function f(){c.uex("ld")}if(e.addEventListener){e.addEventListener("load",f,false)}else{if(e.attachEvent){e.attachEvent("onload",f)}}a.tag=d("tag");a.log=d("log");a.reset=d("rst");c.ue_csm=c;c.ue=a;c.ueLogError=d("err");c.ues=d("ues");c.uet=d("uet");c.uex=d("uex");c.uet("ue")})(window);(function